In [ ]:
# this notebook is just testing how to download data, what data is available, etc.

In [55]:
# !python -m pip install --upgrade pip
# !pip install mindscope_utilities --upgrade
import pandas as pd
import numpy as np
from tqdm import tqdm

from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache
import mindscope_utilities.visual_behavior_ophys as ophys

cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir='data')
# session_table = cache.get_ophys_session_table()
experiment_table = cache.get_ophys_experiment_table()
experiment_table = experiment_table.reset_index()

# from the experiment table we only want those where project_code is VisualBehaviorMultiscope
multiscope_experiment_ids = experiment_table[experiment_table['project_code'] == 'VisualBehaviorMultiscope']['ophys_experiment_id'].tolist()

# reminder: an experiment is a single imaging plane acquired in a single session
# https://allensdk.readthedocs.io/en/latest/visual_behavior_optical_physiology.html

In [46]:
len(multiscope_experiment_ids) # how many multiscope experiments we have

867

In [47]:
# for now let's try only 5
multiscope_experiment_ids = multiscope_experiment_ids[:5]

In [48]:
# this loop downloads behavioural data for each of the experiments whose ids we found earlier and saves them in a dictionary where the key is experiment id
experiments = {}
for id in multiscope_experiment_ids:
  experiments[id] = cache.get_behavior_ophys_experiment(id)

In [49]:
experiments[951980471]
# each experiment is a BehaviorOpysExperiment object
# there is a table at the bottom of this page with description of what such objects contain and how to access that information
# https://allensdk.readthedocs.io/en/v2.12.3/visual_behavior_ophys_nwb.html

In [50]:
# for example this shows rewards data for experiment 951980471
experiments[951980471].rewards

,volume,timestamps,autorewarded
0,0.005,322.91986,True
1,0.005,333.44514,True
2,0.005,364.23697,True
3,0.005,388.25660,True
4,0.005,455.04450,True
...,...,...,...
66,0.007,2956.13799,False
67,0.007,2981.52539,False
68,0.007,2998.07224,False
69,0.007,3138.50361,False


In [51]:
# this shows dff data for experiment 951980471
experiments[951980471].dff_traces.reset_index()

,cell_specimen_id,cell_roi_id,dff
0,1086613265,1080743723,"[0.9365729093551636, 0.5824856162071228, 1.296..."
1,1086613823,1080743752,"[0.555761456489563, 0.7478731870651245, 0.4397..."
2,1086619526,1080743754,"[0.2593991160392761, 0.30149346590042114, 0.34..."
3,1086614149,1080743763,"[0.4112476706504822, 0.1977161169052124, 0.143..."
4,1086614351,1080743765,"[0.08977238088846207, 0.0746675506234169, 0.26..."
5,1086614819,1080743775,"[0.19020046293735504, 0.02105860970914364, 0.1..."
6,1086615201,1080743785,"[0.5737669467926025, 0.4127621352672577, 0.129..."
7,1086615620,1080743794,"[2.900156259536743, 0.9670373201370239, 0.5696..."
8,1086615837,1080743799,"[0.42394280433654785, 0.6050381660461426, 0.64..."
9,1086616206,1080743817,"[0.6248000264167786, 0.13580583035945892, 0.10..."


In [52]:
# metadata available for this experiment (this will be useful for downloading neural data later)
experiments[951980471].metadata

{'age_in_days': 233,
 'behavior_session_id': 951520319,
 'behavior_session_uuid': UUID('592420fb-911d-41c5-8763-4ebdeb0ef416'),
 'cre_line': 'Sst-IRES-Cre',
 'date_of_acquisition': datetime.datetime(2019, 9, 20, 9, 45, 29, tzinfo=tzlocal()),
 'driver_line': ['Sst-IRES-Cre'],
 'emission_lambda': 520.0,
 'equipment_name': 'MESO.1',
 'excitation_lambda': 910.0,
 'experiment_container_id': 1018028342,
 'field_of_view_height': 512,
 'field_of_view_width': 512,
 'full_genotype': 'Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt',
 'imaging_depth': 150,
 'imaging_plane_group': 0,
 'imaging_plane_group_count': 4,
 'indicator': 'GCaMP6f',
 'mouse_id': 457841,
 'ophys_experiment_id': 951980471,
 'ophys_frame_rate': 11.0,
 'ophys_session_id': 951410079,
 'project_code': None,
 'reporter_line': 'Ai148(TIT2L-GC6f-ICL-tTA2)',
 'session_type': 'OPHYS_1_images_A',
 'sex': 'F',
 'stimulus_frame_rate': 60.0,
 'targeted_structure': 'VISp'}

In [53]:
# list of metadata we want to download with out neural data
metadata_keys = ['cre_line', 'imaging_depth', 'mouse_id', 'ophys_experiment_id', 'session_type', 'targeted_structure']

In [56]:
# this downloads the neural data (copied from load_Allen_Visual_Behavior_from_SDK)
neural_data = []
for ophys_experiment_id in tqdm(experiments.keys()):
  this_experiment = experiments[ophys_experiment_id]
  this_experiment_neural_data = ophys.build_tidy_cell_df(this_experiment)

  for metadata_key in metadata_keys:
    this_experiment_neural_data[metadata_key] = this_experiment.metadata[metadata_key]

  # append the data for this experiment to a list
  neural_data.append(this_experiment_neural_data)

# concatate the list of dataframes into a single dataframe
neural_data = pd.concat(neural_data)

100%|██████████| 5/5 [00:02<00:00,  2.02it/s]


In [57]:
neural_data # once we have neural data in a pandas dataframe like this it's much easier to work with it

,timestamps,dff,events,filtered_events,cell_roi_id,cell_specimen_id,cre_line,imaging_depth,mouse_id,ophys_experiment_id,session_type,targeted_structure
0,9.33347,0.178109,0.0,0.0,1080742677,1086611645,Sst-IRES-Cre,300,457841,951980486,OPHYS_1_images_A,VISl
1,9.42667,0.491694,0.0,0.0,1080742677,1086611645,Sst-IRES-Cre,300,457841,951980486,OPHYS_1_images_A,VISl
2,9.51988,0.370677,0.0,0.0,1080742677,1086611645,Sst-IRES-Cre,300,457841,951980486,OPHYS_1_images_A,VISl
3,9.61309,0.052906,0.0,0.0,1080742677,1086611645,Sst-IRES-Cre,300,457841,951980486,OPHYS_1_images_A,VISl
4,9.70629,0.115736,0.0,0.0,1080742677,1086611645,Sst-IRES-Cre,300,457841,951980486,OPHYS_1_images_A,VISl
...,...,...,...,...,...,...,...,...,...,...,...,...
48311,4513.46209,0.137588,0.0,0.0,1080745030,1086620900,Sst-IRES-Cre,75,457841,951980475,OPHYS_1_images_A,VISp
48312,4513.55533,0.167492,0.0,0.0,1080745030,1086620900,Sst-IRES-Cre,75,457841,951980475,OPHYS_1_images_A,VISp
48313,4513.64857,0.100330,0.0,0.0,1080745030,1086620900,Sst-IRES-Cre,75,457841,951980475,OPHYS_1_images_A,VISp
48314,4513.74180,0.197114,0.0,0.0,1080745030,1086620900,Sst-IRES-Cre,75,457841,951980475,OPHYS_1_images_A,VISp


In [72]:
neural_data[neural_data['ophys_experiment_id'] == 951980471] # neural data for one experiment

,timestamps,dff,events,filtered_events,cell_roi_id,cell_specimen_id,cre_line,imaging_depth,mouse_id,ophys_experiment_id,session_type,targeted_structure
0,9.26356,0.936573,0.000000,0.000000,1080743723,1086613265,Sst-IRES-Cre,150,457841,951980471,OPHYS_1_images_A,VISp
1,9.35677,0.582486,0.000000,0.000000,1080743723,1086613265,Sst-IRES-Cre,150,457841,951980471,OPHYS_1_images_A,VISp
2,9.44998,1.296005,0.556873,0.400762,1080743723,1086613265,Sst-IRES-Cre,150,457841,951980471,OPHYS_1_images_A,VISp
3,9.54318,0.844898,0.000000,0.148502,1080743723,1086613265,Sst-IRES-Cre,150,457841,951980471,OPHYS_1_images_A,VISp
4,9.63639,1.181188,0.467264,0.343829,1080743723,1086613265,Sst-IRES-Cre,150,457841,951980471,OPHYS_1_images_A,VISp
...,...,...,...,...,...,...,...,...,...,...,...,...
48311,4513.43879,1.033316,0.499144,0.359216,1080743847,1086616398,Sst-IRES-Cre,150,457841,951980471,OPHYS_1_images_A,VISp
48312,4513.53202,1.681890,0.787122,0.699572,1080743847,1086616398,Sst-IRES-Cre,150,457841,951980471,OPHYS_1_images_A,VISp
48313,4513.62526,2.074159,0.691039,0.713992,1080743847,1086616398,Sst-IRES-Cre,150,457841,951980471,OPHYS_1_images_A,VISp
48314,4513.71849,1.668930,0.000000,0.195007,1080743847,1086616398,Sst-IRES-Cre,150,457841,951980471,OPHYS_1_images_A,VISp


In [62]:
stimulus_presentations_one_experiment = experiments[951980471].stimulus_presentations.reset_index().drop(columns = ['image_set'])
stimulus_presentations_one_experiment.insert(0, 'ophys_experiment_id', [951980471 for i in range(stimulus_presentations_one_experiment.shape[0])]) # adds column with experiment id
stimulus_presentations_one_experiment

,ophys_experiment_id,stimulus_presentations_id,start_time,stop_time,duration,image_name,image_index,is_change,omitted,start_frame,end_frame
0,951980471,0,309.27537,309.52557,0.25020,im065,0,False,False,17986,18001.0
1,951980471,1,310.02598,310.27619,0.25021,im065,0,False,False,18031,18046.0
2,951980471,2,310.77660,311.02680,0.25020,im065,0,False,False,18076,18091.0
3,951980471,3,311.52721,311.77740,0.25019,im065,0,False,False,18121,18136.0
4,951980471,4,312.27782,312.52806,0.25024,im065,0,False,False,18166,18181.0
...,...,...,...,...,...,...,...,...,...,...,...
4796,951980471,4796,3909.81737,3910.06757,0.25020,im065,0,False,False,233842,233857.0
4797,951980471,4797,3910.56798,3910.81819,0.25021,im065,0,False,False,233887,233902.0
4798,951980471,4798,3911.31860,3911.56880,0.25020,im065,0,False,False,233932,233947.0
4799,951980471,4799,3912.06921,3912.31939,0.25018,im065,0,False,False,233977,233992.0


In [70]:
experiments[951980471].eye_tracking.reset_index() # eyes data from one experiment but from this we probably only need pupil_area

,frame,timestamps,cr_area,eye_area,pupil_area,likely_blink,pupil_area_raw,cr_area_raw,eye_area_raw,cr_center_x,...,eye_center_x,eye_center_y,eye_width,eye_height,eye_phi,pupil_center_x,pupil_center_y,pupil_width,pupil_height,pupil_phi
0,0,0.15829,183.586742,63999.056889,12139.327201,False,12139.327201,183.586742,63999.056889,346.607136,...,359.155663,206.109419,159.882116,127.415955,-0.104048,324.758778,217.811200,62.161627,61.233369,-0.537537
1,1,0.17177,183.453405,63878.120921,12255.297397,False,12255.297397,183.453405,63878.120921,346.946852,...,359.295294,205.999745,160.111177,126.993242,-0.109490,324.781277,218.094581,62.457844,61.034058,-0.507705
2,2,0.18748,176.245497,63975.675600,12128.875269,False,12128.875269,176.245497,63975.675600,346.735257,...,358.987745,206.186120,159.889628,127.363421,-0.110130,325.287244,217.632501,62.134861,61.513908,0.731344
3,3,0.20904,182.139778,63997.358050,12394.953413,False,12394.953413,182.139778,63997.358050,346.869934,...,359.471260,206.022686,159.963601,127.347669,-0.106686,325.115191,218.329534,62.812707,60.753917,-0.113071
4,4,0.24243,186.747323,64046.312897,12436.578877,False,12436.578877,186.747323,64046.312897,346.672515,...,359.413554,206.041874,160.226985,127.235588,-0.112555,324.953067,217.859342,62.918090,59.972431,-0.252311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135800,135800,4534.03595,NaN,NaN,NaN,True,61890.929423,102.772064,67905.572438,341.548991,...,356.858921,209.137332,161.523517,133.819616,-0.158598,411.293800,210.587884,140.358451,79.629106,-0.465248
135801,135801,4534.05648,NaN,NaN,NaN,True,33070.886867,102.398486,67718.266973,341.085894,...,356.888498,208.115662,161.162372,133.749544,-0.174015,358.045252,227.244440,72.704263,102.600147,0.092584
135802,135802,4534.09265,NaN,NaN,NaN,True,34258.357816,103.765405,67789.097106,341.066068,...,357.557303,207.375645,160.867255,134.135066,-0.157242,344.243840,216.308654,89.993688,104.425926,-0.364177
135803,135803,4534.12325,NaN,NaN,NaN,True,32143.878736,104.025535,67620.109455,341.309446,...,358.153603,207.694102,159.907653,134.603622,-0.148380,352.524524,219.959223,83.036551,101.151937,-0.078646


In [80]:
experiments[951980471].trials.reset_index().columns 
# this one has information about rewards (time and volume) also the type of trial (hit, miss, false alarm, correct rejection)
# don't know if we need it but we know it's available too

Index(['trials_id', 'start_time', 'stop_time', 'lick_times', 'reward_time',
       'reward_volume', 'hit', 'false_alarm', 'miss', 'stimulus_change',
       'aborted', 'go', 'catch', 'auto_rewarded', 'correct_reject',
       'trial_length', 'response_time', 'change_frame', 'change_time',
       'response_latency', 'initial_image_name', 'change_image_name'],
      dtype='object')

In [ ]:
# So in general we can actually get all the data in nice data frames but it will require some processing (joining, etc.)